In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

import pandas as pd
import numpy as np

In [2]:
file_path = "../Nelson/HOF_stats.csv"

df = pd.read_csv(file_path)

df = df.copy()

df.head()
# if the season doesn't contain a -, delete the row



,Unnamed: 0,2P,2P%,2PA,3P,3P%,3PA,AST,Age,BLK,...,PF,PTS,Player,Pos,STL,Season,TOV,TRB,Tm,eFG%
0,0,NaN,NaN,NaN,NaN,NaN,NaN,51.0,24.0,NaN,...,210.0,588.0,Wes Unseld,C,NaN,1953-54,NaN,419.0,MNL,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,100.0,25.0,NaN,...,262.0,1311.0,Wes Unseld,C,NaN,1954-55,NaN,802.0,MNL,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,164.0,26.0,NaN,...,245.0,1526.0,Wes Unseld,C,NaN,1955-56,NaN,992.0,MNL,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,139.0,27.0,NaN,...,251.0,1434.0,Wes Unseld,C,NaN,1956-57,NaN,932.0,MNL,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,134.0,28.0,NaN,...,236.0,1659.0,Wes Unseld,C,NaN,1957-58,NaN,862.0,CIN,NaN


In [3]:
# if the season doesn't contain a -, delete the row
# OR filter retain the rows where the - exists

# df["Season"].drop(df["Season"].str.contains('nan'))
df = df[df["Season"].notnull()]
df = df[df["Season"].str.contains('-')]
df
# df["Season"] = df["Season"].dropna(how=any)
# df["Season"] = str(df["Season"])
# df.loc[df["Season"].str.contains('-')]

,Unnamed: 0,2P,2P%,2PA,3P,3P%,3PA,AST,Age,BLK,...,PF,PTS,Player,Pos,STL,Season,TOV,TRB,Tm,eFG%
0,0,NaN,NaN,NaN,NaN,NaN,NaN,51.0,24.0,NaN,...,210.0,588.0,Wes Unseld,C,NaN,1953-54,NaN,419.0,MNL,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,100.0,25.0,NaN,...,262.0,1311.0,Wes Unseld,C,NaN,1954-55,NaN,802.0,MNL,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,164.0,26.0,NaN,...,245.0,1526.0,Wes Unseld,C,NaN,1955-56,NaN,992.0,MNL,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,139.0,27.0,NaN,...,251.0,1434.0,Wes Unseld,C,NaN,1956-57,NaN,932.0,MNL,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,134.0,28.0,NaN,...,236.0,1659.0,Wes Unseld,C,NaN,1957-58,NaN,862.0,CIN,NaN
5,5,NaN,NaN,NaN,NaN,NaN,NaN,91.0,29.0,NaN,...,216.0,1009.0,Wes Unseld,C,NaN,1958-59,NaN,605.0,STL,NaN
6,6,NaN,NaN,NaN,NaN,NaN,NaN,127.0,30.0,NaN,...,248.0,1416.0,Wes Unseld,C,NaN,1959-60,NaN,721.0,STL,NaN
7,7,NaN,NaN,NaN,NaN,NaN,NaN,172.0,31.0,NaN,...,248.0,1471.0,Wes Unseld,C,NaN,1960-61,NaN,677.0,STL,NaN
8,8,NaN,NaN,NaN,NaN,NaN,NaN,68.0,32.0,NaN,...,136.0,837.0,Wes Unseld,C,NaN,1961-62,NaN,350.0,STL,NaN
9,9,NaN,NaN,NaN,NaN,NaN,NaN,27.0,33.0,NaN,...,137.0,395.0,Wes Unseld,C,NaN,1962-63,NaN,177.0,BOS,NaN


In [6]:
engine = create_engine("sqlite:///NBA.sqlite")

df.to_sql(name='hall_of_famers', con=engine, if_exists='replace', index=False)

In [7]:
pd.read_sql_query('SELECT * FROM hall_of_famers', con=engine).head()

,Unnamed: 0,2P,2P%,2PA,3P,3P%,3PA,AST,Age,BLK,...,PF,PTS,Player,Pos,STL,Season,TOV,TRB,Tm,eFG%
0,0,NaN,NaN,NaN,NaN,NaN,NaN,51.0,24.0,NaN,...,210.0,588.0,Wes Unseld,C,NaN,1953-54,NaN,419.0,MNL,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,100.0,25.0,NaN,...,262.0,1311.0,Wes Unseld,C,NaN,1954-55,NaN,802.0,MNL,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,164.0,26.0,NaN,...,245.0,1526.0,Wes Unseld,C,NaN,1955-56,NaN,992.0,MNL,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,139.0,27.0,NaN,...,251.0,1434.0,Wes Unseld,C,NaN,1956-57,NaN,932.0,MNL,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,134.0,28.0,NaN,...,236.0,1659.0,Wes Unseld,C,NaN,1957-58,NaN,862.0,CIN,NaN
